<a href="https://www.kaggle.com/code/okeoluwanifemi/oke-working-book?scriptVersionId=296632681" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Oke working book

In [1]:
#Installing libries 
!pip install -q -U bitsandbytes accelerate transformers sentencepiece protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 36.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
onnx 1.20.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
ray 2.52.1 requires click!=8.3.*,>=7.0, but you have click 8.3.1 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, b

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import sys
from io import StringIO
import math
import numpy as np
import warnings
import os
import inspect
from multiprocessing import Process, Queue
import time
import torch

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]= "python"

In [3]:
model_path = "/kaggle/input/llama-3/transformers/8b-hf/1"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # Better for math
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
)

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [6]:
TIME_LIMIT = 60 

class TimeoutException(Exception): pass
def timeout_handler(signum, frame): raise TimeoutException

def solve_problem(problem_text):
    
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a Python expert.<|eot_id|><|start_header_id|>user<|end_header_id|>

Problem: {problem_text}

Write Python code to solve this using brute force search.

Example approach:
````python
# Try all possible values
for alice_age in range(1, 100):
    for bob_age in range(1, 100):
        for alice_sweets in range(1, 100):
            for bob_sweets in range(1, 100):
                # Check condition 1: sum constraint after adding ages
                if (alice_age + alice_sweets) == 2 * (bob_age + bob_sweets):
                    # Check condition 2: product constraint
                    if (alice_age * alice_sweets) == 4 * (bob_age * bob_sweets):
                        # Check condition 3: after giving 5 sweets
                        new_alice = alice_sweets - 5
                        new_bob = bob_sweets + 5
                        if (alice_age + new_alice) == (bob_age + new_bob):
                            if (alice_age * new_alice) == (bob_age * new_bob):
                                answer = alice_age * bob_age
                                print(f"ANSWER: {{answer}}")
                                exit()
````

Now write similar code for this problem. Use the EXACT same structure:
- nested for loops
- check each condition with if statements
- print ANSWER: followed by the number
- call exit() when found

Code:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
````python"""

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=800,
                temperature=0.4,  # Lower temp for more deterministic code
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        
        # Extract code (everything before closing ```)
        code = response.split('```')[0].strip()
        
        # Remove any remaining markdown
        code = re.sub(r'^python\s*', '', code, flags=re.MULTILINE)
        
        print("="*70)
        print("GENERATED CODE:")
        print(code)
        print("="*70)
        
        # Execute with timeout
        import signal
        
        def timeout_handler(signum, frame):
            raise TimeoutError("Code execution timeout")
        
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(45)  # 45 second timeout
        
        old_stdout = sys.stdout
        sys.stdout = captured = StringIO()
        
        try:
            exec_globals = {'exit': lambda: None}  # Prevent actual exit
            exec(code, exec_globals)
            
        except SystemExit:
            pass
        except TimeoutError:
            print("TIMEOUT")
        except TimeoutException:
            sys.stdout = old_stdout
            print("⏰ TIMEOUT (Loops too big)")
            return 0
        except Exception as e:
            print(f"ERROR: {e}")
        finally:
            signal.alarm(0)
            sys.stdout = old_stdout
            
        output = captured.getvalue()
        
        print("CODE OUTPUT:")
        print(output if output else "(no output)")
        print("="*70)
        
        # Extract answer
        match = re.search(r'ANSWER:\s*(\d+)', output, re.IGNORECASE)
        if match:
            return int(match.group(1))
        
        # Fallback: any number in output
        nums = re.findall(r'\b(\d+)\b', output)
        if nums:
            return int(nums[-1])
        
        return 0
        
    except Exception as e:
        print(f"Error: {e}")
        return 0


# Test
problem1 = """Alice and Bob are each holding some integer number of sweets. Alice says to Bob: "If
we each added the number of sweets we're holding to our (positive integer) age, my answer would
be double yours. If we took the product, then my answer would be four times yours." Bob replies:
"Why don't you give me five of your sweets because then both our sum and product would be equal."
What is the product of Alice and Bob's ages?"""

print("TESTING PROBLEM 1")
print("="*70)
result = solve_problem(problem1)
print(f"\n FINAL ANSWER: {result}")
print(f" Expected: 50")
print(f"Match: {'YES' if result == 50 else 'NO'}")

TESTING PROBLEM 1
GENERATED CODE:
# Try all possible values
for alice_age in range(1, 100):
    for bob_age in range(1, 100):
        for alice_sweets in range(1, 100):
            for bob_sweets in range(1, 100):
                # Check condition 1: sum constraint after adding ages
                if (alice_age + alice_sweets) == 2 * (bob_age + bob_sweets):
                    # Check condition 2: product constraint
                    if (alice_age * alice_sweets) == 4 * (bob_age * bob_sweets):
                        # Check condition 3: after giving 5 sweets
                        new_alice = alice_sweets - 5
                        new_bob = bob_sweets + 5
                        if (alice_age + new_alice) == (bob_age + new_bob):
                            if (alice_age * new_alice) == (bob_age * new_bob):
                                answer = alice_age * bob_age
                                print(f"ANSWER: {answer}")
                                exit()
CODE OUTPUT:
ANS

In [7]:
problem2 = """A 500 by 500 square is divided into k rectangles, each having integer side lengths. Given
that no two of these rectangles have the same perimeter, the largest possible value of k is K. What
is the remainder when K is divided by 10^5 ?"""

print("\n" + "="*70)
print("TESTING PROBLEM 2")
print("="*70)
result = solve_problem(problem2)
print(f"\n FINAL ANSWER: {result}")
print(f" Expected: 520")
print(f"Match: {'YES' if result == 520 else 'NO'}")


TESTING PROBLEM 2
GENERATED CODE:
# Try all possible values
for alice_age in range(1, 100):
    for bob_age in range(1, 100):
        for alice_sweets in range(1, 100):
            for bob_sweets in range(1, 100):
                # Check condition 1: sum constraint after adding ages
                if (alice_age + alice_sweets) == 2 * (bob_age + bob_sweets):
                    # Check condition 2: product constraint
                    if (alice_age * alice_sweets) == 4 * (bob_age * bob_sweets):
                        # Check condition 3: after giving 5 sweets
                        new_alice = alice_sweets - 5
                        new_bob = bob_sweets + 5
                        if (alice_age + new_alice) == (bob_age + new_bob):
                            if (alice_age * new_alice) == (bob_age * new_bob):
                                answer = alice_age * bob_age
                                print(f"ANSWER: {answer}")
                                exit()
CODE OUTPUT:
AN

In [8]:
problem3 = """A tournament is held with 2^20 runners each of which has a different running speed. In
each race, two runners compete against each other with the faster runner always winning the race.
The competition consists of 20 rounds with each runner starting with a score of 0. In each round,
the runners are paired in such a way that in each pair, both runners have the same score at the
beginning of the round. The winner of each race in the i^th round receives 2^(20−i) points and the loser
gets no points.
At the end of the tournament, we rank the competitors according to their scores. Let N denote the
number of possible orderings of the competitors at the end of the tournament. Let k be the largest
positive integer such that 10^k divides N. What is the remainder when k is divided by 10^5
?"""
print("\n" + "="*70)
print("TESTING PROBLEM 3")
print("="*70)
result = solve_problem(problem3)
print(f"\n FINAL ANSWER: {result}")
print(f" Expected: 21818")
print(f"Match: {'YES' if result == 21818 else 'NO'}")


TESTING PROBLEM 3
GENERATED CODE:
def find_k():
    # YOUR CODE HERE
    # Use the EXACT same structure
    # - nested for loops
    # - check each condition with if statements
    # - print ANSWER: followed by the number
    # - call exit() when found
    pass

if __name__ == '__main__':
    find_k()
CODE OUTPUT:
(no output)

 FINAL ANSWER: 0
 Expected: 21818
Match: NO


In [9]:
problem4 = """
Let ABC be an acute-angled triangle with integer side lengths and AB < AC.
Points D and E lie on segments BC and AC, respectively, such that AD = AE = AB.
Line AB intersects line DE at X. Circles B X D and CED intersect for the second time at Y.
Suppose that X is on line AB, Y is on segment BC, and AC = 5.
Find the remainder when the value of BC multiplied by AB is divided by 10^5.
"""


print("\n" + "="*70)
print("TESTING PROBLEM 4")
print("="*70)
result = solve_problem(problem4)
print(f"\n FINAL ANSWER: {result}")
print(f" Expected: 336")
print(f"Match: {'YES' if result == 336 else 'NO'}")


TESTING PROBLEM 4
GENERATED CODE:
# Try all possible values
for alice_age in range(1, 100):
    for bob_age in range(1, 100):
        for alice_sweets in range(1, 100):
            for bob_sweets in range(1, 100):
                # Check condition 1: sum constraint after adding ages
                if (alice_age + alice_sweets) == 2 * (bob_age + bob_sweets):
                    # Check condition 2: product constraint
                    if (alice_age * alice_sweets) == 4 * (bob_age * bob_sweets):
                        # Check condition 3: after giving 5 sweets
                        new_alice = alice_sweets - 5
                        new_bob = bob_sweets + 5
                        if (alice_age + new_alice) == (bob_age + new_bob):
                            if (alice_age * new_alice) == (bob_age * new_bob):
                                answer = alice_age * bob_age
                                print(f"ANSWER: {answer}")
                                exit()
CODE OUTPUT:
AN

In [10]:
problem5 = """
Let f : Z_{>=1} -> Z_{>=1} be a function such that for all positive integers m and n,
f(m) + f(n) = f(m + n + mn).

Across all functions f such that f(n) <= 1000 for all n <= 1000,
how many different values can f(2024) take?
"""

print("\n" + "="*70)
print("TESTING PROBLEM 5")
print("="*70)
result = solve_problem(problem5)
print(f"\n FINAL ANSWER: {result}")
print(f" Expected: 580")
print(f"Match: {'YES' if result == 580 else 'NO'}")


TESTING PROBLEM 5
GENERATED CODE:
# Try all possible values
for alice_age in range(1, 1000):
    for bob_age in range(1, 1000):
        for alice_sweets in range(1, 1000):
            for bob_sweets in range(1, 1000):
                # Check condition 1: sum constraint after adding ages
                if (alice_age + alice_sweets) == 2 * (bob_age + bob_sweets):
                    # Check condition 2: product constraint
                    if (alice_age * alice_sweets) == 4 * (bob_age * bob_sweets):
                        # Check condition 3: after giving 5 sweets
                        new_alice = alice_sweets - 5
                        new_bob = bob_sweets + 5
                        if (alice_age + new_alice) == (bob_age + new_bob):
                            if (alice_age * new_alice) == (bob_age * new_bob):
                                answer = alice_age * bob_age
                                print(f"ANSWER: {answer}")
                                exit()
CODE OUTPUT